In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

# Imports

In [ ]:
import os
import sys
sys.path.append('/home/rcendre/classification')

import itertools
import pandas
import webbrowser
from pathlib import Path
import matplotlib.pyplot as plt
from misvm import SIL, MISVM
from numpy import array, logspace
from scipy.stats import randint,uniform
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import davies_bouldin_score
from toolbox.classification.common import Data, Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.models.models import CustomMIL, MultimodalClassifier
from toolbox.models.builtin import Applications
from toolbox.IO import dermatology
from sklearn.metrics import f1_score
from toolbox.transforms.common import PredictorTransform
from toolbox.transforms.labels import OrderedEncoder
from toolbox.transforms.images import DistributionImageTransform, DWTImageTransform, FourierImageTransform, HaralickImageTransform, SpatialImageTransform
from toolbox.views.common import Views, ViewsTools
from toolbox.views.images import ImagesViews
import warnings
warnings.filterwarnings('ignore') 

# Parameters

In [ ]:
# Advanced parameters
data_type='Full'
validation = 4
settings = Settings.get_default_dermatology()

# Photography

In [ ]:
p_inputs = IO.load('Photography.pickle')

In [ ]:
extractor = 'InceptionResNetV2Avg'

# SVM Linear
model = Pipeline([('scale', MinMaxScaler()), ('clf', SVC(kernel='linear', class_weight='balanced', probability=True))])
model_params = {'clf__C': logspace(-2, 3, 6).tolist()}

In [ ]:
pred_folds = [([1,2], [3]),
             ([2,3], [4]),
             ([3,4], [1]),
             ([4,1], [2])]
Tools.evaluate(p_inputs, {'datum': extractor, 'label_encode': 'LesionEncode'}, model, 'Prediction', folds=pred_folds, distribution=model_params) 

In [ ]:
from IPython.display import HTML
from IPython.display import display

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))     

name = f'Prediction'
# Label
display(HTML(ViewsTools.dataframe_renderer([Views.report(p_inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Photography'])))

# Dermoscopy

In [ ]:
d_inputs = IO.load('Dermoscopy.pickle')

In [ ]:
extractor = 'InceptionResNetV2Avg'

# SVM Linear
model = Pipeline([('scale', MinMaxScaler()), ('clf', SVC(kernel='linear', class_weight='balanced', probability=True))])
model_params = {'clf__C': logspace(-2, 3, 6).tolist()}

In [ ]:
pred_folds = [([1,2], [3]),
             ([2,3], [4]),
             ([3,4], [1]),
             ([4,1], [2])]
Tools.evaluate(d_inputs, {'datum': extractor, 'label_encode': 'LesionEncode'}, model, 'Prediction', folds=pred_folds, distribution=model_params) 

In [ ]:
from IPython.display import HTML
from IPython.display import display

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))     

name = f'Prediction'
# Label
display(HTML(ViewsTools.dataframe_renderer([Views.report(d_inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Dermoscopy'])))

# Microscopy

In [ ]:
m_inputs = IO.load('Microscopy.pickle')

In [ ]:
extractor = 'ResNetAvg'
low_folds = [([1], [2]),
             ([2], [3]),
             ([3], [4]),
             ([4], [1])]
Tools.fit_predict(m_inputs, {'datum': extractor, 'label_encode': 'LesionEncode'}, MinMaxScaler(), 'Scaled', folds=low_folds)

In [ ]:
all_image = [True] * len(m_inputs.index)
single_image = m_inputs['ID_Image'] == '0M'
Data.build_bags(m_inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', extractor)
Data.build_bags(m_inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', 'Scaled')
m_inputs = m_inputs[single_image].reset_index()

In [ ]:
pred_folds = [([2], [3]),
                ([3], [4]),
                ([4], [1]),
                ([1], [2])]
model = CustomMIL(MISVM(kernel='linear', C=1.0, verbose=False))
model_params = {'estimator__C': logspace(-2, 3, 6).tolist()}
Tools.fit_predict(m_inputs, {'datum': 'Scaled', 'label_encode': 'LesionEncode'}, model, 'Prediction', folds=pred_folds, distribution=model_params)

In [ ]:
from IPython.display import HTML
from IPython.display import display

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))     

name = f'Prediction'
# Label
display(HTML(ViewsTools.dataframe_renderer([Views.report(m_inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Microscopy'])))

# Fusion

In [ ]:
inputs = pandas.concat([p_inputs, d_inputs, m_inputs], axis=0)
 # Save
IO.save(inputs, 'Low_Sequential.pickle')

# Sequential

In [ ]:
inputs = IO.load('Low_Sequential.pickle')

In [ ]:
all_image = [True] * len(inputs.index)
single_image = inputs['Modality'] == 'Photography'
Data.build_bags(inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', f'Prediction_{Tools.PROBABILITY}')
inputs = inputs[single_image].reset_index()

In [ ]:
high_folds = [([3],[1]),
             ([4],[2]),
             ([1],[3]),
             ([2],[4])]
single = MultimodalClassifier(method='single', metric=f1_score)
individual = MultimodalClassifier(method='individual', metric=f1_score)

In [ ]:
Tools.evaluate(inputs, {'datum': f'Prediction_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, single, 'Single')
Tools.evaluate(inputs, {'datum': f'Prediction_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, individual, 'Individual') 

In [ ]:
IO.save(inputs, 'High_Sequential.pickle')

# Score

In [ ]:
from IPython.display import HTML
from IPython.display import display

inputs = IO.load('High_Sequential.pickle')

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))     

name = f'Single'
# Label
display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                    title=[f'Test - {name}'])))

name = f'Individual'
display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                        title=[f'Test - {name}'])))